# Spatial Databases

In [3]:
# This code cell starts the necessary setup for Hour of CI lesson notebooks.
# First, it enables users to hide and unhide code by producing a 'Toggle raw code' button below.
# Second, it imports the hourofci package, which is necessary for lessons and interactive Jupyter Widgets.
# Third, it helps hide/control other aspects of Jupyter Notebooks to improve the user experience
# This is an initialization cell
# It is not displayed because the Slide Type is 'Skip'

from IPython.display import HTML, IFrame, Javascript, display
from ipywidgets import interactive
import ipywidgets as widgets
from ipywidgets import Layout

import getpass # This library allows us to get the username (User agent string)

# import package for hourofci project
import sys
sys.path.append('../../supplementary') # relative path (may change depending on the location of the lesson notebook)
# sys.path.append('supplementary')
import hourofci
try:
    import os
    os.chdir('supplementary')
except:
    pass

# load javascript to initialize/hide cells, get user agent string, and hide output indicator
# hide code by introducing a toggle button "Toggle raw code"
HTML(''' 
    <script type="text/javascript" src=\"../../supplementary/js/custom.js\"></script>
    
    <style>
        .output_prompt{opacity:0;}
    </style>
    
    <input id="toggle_code" type="button" value="Toggle raw code">
''')

In the last chapter, from the Kindergarten and Schools example we saw that, with traditional database and SQL queries we can't ask any **"spatially relevant"** questions nor we can identify any spatial patterns (or simply we can't map). 

To facilitate such questions we should some how **spatially enable our database**. This is exactly what a spatial database is build for. Let's look at the formal definition for a spatial database

>A **spatial database is a database** that has been **enhanced to include spatial data that represents objects defined in a geometric space, along with tools for querying and analyzing such data**.

The first key point to note here is that **spatial database is a database**, so we can still leverage all the functionalities of a traditional non-spatial database. The second point is the capability of spatial database to include a **new type called Geometry** and perform operations on them and between them. Let's look at three example relations (tables) that has spatial data in the form of geometry. 

![geometry_types](supplementary/images/geometry_types.PNG)

As you can see the three tables have a special column called **geometry** (the images shown in geometry column are for representational purpose). The three basic geometry types are 

1. Points (schools, shooting, earthquake,your location)
2. Lines (rivers, streets, roads, railway lines)
3. Polygon (countries, states, census tracts, zip codes)

And then there other geometries that are build on top of the basic geometry types such as 
Multipoint, Multiline, and MultiPolygons.

Apart from supporting geometry types, spatial database also supports operations on geometry as well as between geometries.

Queries that involve geometry types are called spatial queries which we are going to cover in the next section. 

## Spatial Queries

> **Spatial queries are queries in a spatial database** that can be answered on the **basis of geometric information only, i.e., the spatial position and extent of the objects involved**.

Let's look at various types of spatial functions and queries

### Containment Query

The function **st_contains(geometry A,geometry B)** returns true if geometry A completely contains geometry B

![Contains_explanation](supplementary/images/containment_detail.png) 

Some real world examples include

![PointInPolygon](supplementary/images/PointInPolygon.png) 

![Covid Cases](supplementary/images/Covid_Cases.png) 

**How many starbucks are there in my state**

![Starbucks](supplementary/images/137-44505.png) 

Lets look at the two **tables** involved in this query

In [8]:
from ipywidgets import Button, HBox, VBox,widgets,Layout,GridspecLayout,IntSlider,HTML
from IPython.display import display
import spatialite
import pandas as pd
db = spatialite.connect('databases/spatialDB.sqlite')
table1 = pd.read_sql_query('select statefp,name,geom as geometry from us_states limit 5',db)
table2 = pd.read_sql_query('select pk_uid,fid,geom as geometry from starbucks limit 5',db)
table1_disp = widgets.Output()
table2_disp = widgets.Output()
table1_header = widgets.HTML(value = f"<b><font color='red'><center>US_STATES</center></b>")
table2_header = widgets.HTML(value = f"<b><font color='red'><center>STARBUCKS</center></b>")
with table1_disp:
    display(table1)
with table2_disp:
    display(table2)
out=HBox([VBox([table1_header,table1_disp],layout = Layout(margin='0 100px 0 0')),VBox([table2_header,table2_disp])])
out

LoadExtensionError: Failed to load SpatiaLite extension. Verify that your python module sqlite3 has load_extension support and check that libspatialite is installed. Tried extension names: mod_spatialite, mod_spatialite.so, mod_spatialite.dylib

As you can see for both the tables (US_STATES and STARBUCKS) there is a geometry column which can be used for spatial querying. Suppose your state is Califronia, then to count all the STARBUCKS that are **within** the state of 'Califorina' we can use the query,

```sql
select count(*) as total_starbucks from us_states u,starbucks s where u.name = 'California' and st_contains(u.geom,s.geom)
```

From the last chapter you might recall that this is a join operation involving multiple tables. But unlike the examples the we have seen there is no explicit key-based relationship between the two tables. 

So instead of using a key-based relationship for the join we are using the relationship between the geometries of the two tables for the join. Such type of joins are called **Spatial Joins**.

>**Spatial Joins** - **Joins attributes from one table to another based on the spatial relationship**.

The clause
```sql
where u.name = 'California' 
```
retrieve all rows from the us_states table with a name 'California' (which is only one)
and the clause
```sql
st_contains(u.geom,s.geom)
```

retrieves those rows from starbucks and us_states table where the **geometry in starbucks table (which is point in case of starbucks table) is contained by the geometry (which is Polygon) in us_states table** (which in this case is California) and then
```sql
count(*) as total_starbucks
```
counts the number of rows returned as a result of the where clause and assign it a name total_starbucks

Let's look at an interactive example of this query. Here we can change the state interactively and get the counts. Along with the counts the locations of the starbucks are also displayed. 

In [ ]:
from ipyleaflet import Map, DrawControl,GeoData,LayerGroup,WidgetControl,Rectangle,basemap_to_tiles,basemaps,Polygon,GeoJSON,Choropleth
from ipywidgets import Button, HBox, VBox,widgets,Layout,GridspecLayout,IntSlider,HTML
from IPython.display import display
import spatialite
import pandas as pd
import geopandas as gpd
import json
import time
db = spatialite.connect('databases/spatialDB.sqlite')
import time
def stateChanged(slid):
    layer_group.clear_layers()
    stateGeomSql = f"SELECT ST_AsBinary(geom) as geom FROM us_states where name='{states.value}';"
    starbucksSql = f"""SELECT ST_AsBinary(s.geom) as geom FROM us_states u,starbucks s where u.name='{states.value}'
     and st_contains(u.geom,s.geom) and s.rowid in(SELECT ROWID 
    FROM SpatialIndex
    WHERE f_table_name = 'starbucks' 
        AND search_frame = u.geom)"""
    gdf = gpd.GeoDataFrame.from_postgis(stateGeomSql, db,crs = 'EPSG:4269').to_crs('EPSG:4326')
    starbucksgdf = gpd.GeoDataFrame.from_postgis(starbucksSql, db,crs = 'EPSG:4326')
    center = [gdf.centroid.y.values[0],gdf.centroid.x.values[0]]
    sMap.center = center
    sMap.zoom = 6
    geo_data = GeoJSON(data = json.loads(gdf.to_json()),style={'opacity': 1, 'dashArray': '9', 'fillOpacity': 0, 'weight': 1})
    layer_group.add_layer(geo_data)
    geo_data_starbucks = GeoJSON(data = json.loads(starbucksgdf.to_json()))
    layer_group.add_layer(geo_data_starbucks)
    counts.value = str(len(starbucksgdf))
    
sql = "SELECT name FROM us_states order by name;"
statedf = pd.read_sql_query(sql,db)
sMap= Map(center=(41.482222, -81.669722), zoom=15,prefer_canvas =True)
layer_group = LayerGroup()
sMap.add_layer(layer_group)
states = widgets.Dropdown(
    options=statedf.name.values,
    value=statedf.name.values[0],
    description='State:',
    disabled=False,
)
counts=widgets.Text(
    value='',
    placeholder='',
    description='Total:',
    disabled=True,
)
states.observe(stateChanged, 'value')
filterParams=HBox([sMap,VBox([states,counts])])
stateChanged(None)
filterParams

We can also modify the question as, **"How many starbucks in each state?"**

![PointInManyPolygons](supplementary/images/PointInMultiplePolygons.png) 

**Total starbucks in each state**

The key difference here is that we are not selecting any particular state and we want our results to be **grouped** by each state name

We can write this **query** as 

```sql
select u.name,count(*) as total_starbucks from us_states u,starbucks s where st_contains(u.geom,s.geom) group by u.name
```

If you compare this query to the previous one you can notice that the clause that checks the state name is removed. 

In [ ]:
from ipywidgets import Button, HBox, VBox,widgets,Layout,GridspecLayout,IntSlider,HTML
from IPython.display import display
db = spatialite.connect('databases/spatialDB.sqlite')
table1 = pd.read_sql_query("""SELECT statefp,name,u.geom as geometry,s.pk_uid,fid,s.geom as geometry from us_states u,starbucks s
 where st_contains(u.geom,s.geom) and s.rowid in(SELECT ROWID 
    FROM SpatialIndex
    WHERE f_table_name = 'starbucks' 
        AND search_frame = u.geom) limit 10""",db)
table1_disp = widgets.Output()
table1_header = widgets.HTML(value = f"<b><font color='red'><center>Matching Rows</center></b>")
with table1_disp:
    display(table1)
out=HBox([VBox([table1_header,table1_disp])])
out

Now if you want to group this table based on name and show the total count for each name use count() and group by name

In [ ]:
disp = widgets.Output()
db = spatialite.connect('databases/spatialDB.sqlite')
stateGeomSql = f"""SELECT u.name,count(*) as total_sbucks from us_states u,starbucks s
 where st_contains(u.geom,s.geom) and s.rowid in(SELECT ROWID 
    FROM SpatialIndex
    WHERE f_table_name = 'starbucks' 
        AND search_frame = u.geom) group by u.name"""
data = pd.read_sql_query(stateGeomSql,con=db)
with disp:
    display(data)
disp

We can ask similar questions using the containment query. 

Try out the examples given below

**How many dominos are there in my state**

Let's look at the tables.

In [ ]:
from ipywidgets import Button, HBox, VBox,widgets,Layout,GridspecLayout,IntSlider,HTML
from IPython.display import display
db = spatialite.connect('databases/spatialDB.sqlite')
table1 = pd.read_sql_query('select statefp,name,geom as geometry from us_states limit 5',db)
table2 = pd.read_sql_query('select pk_uid,fid,geom as geometry from dominos limit 5',db)
table1_disp = widgets.Output()
table2_disp = widgets.Output()
table1_header = widgets.HTML(value = f"<b><font color='red'><center>US_STATES</center></b>")
table2_header = widgets.HTML(value = f"<b><font color='red'><center>DOMINOS</center></b>")
with table1_disp:
    display(table1)
with table2_disp:
    display(table2)
out=HBox([VBox([table1_header,table1_disp],layout = Layout(margin='0 100px 0 0')),VBox([table2_header,table2_disp])])
out

And this is the query

```sql
SELECT ST_AsBinary(s.geom) as geom FROM us_states u,dominos s where u.name='California'
     and st_contains(u.geom,s.geom)
```

Let's look at an interactive example

In [ ]:
from ipyleaflet import Map, DrawControl,GeoData,LayerGroup,WidgetControl,Rectangle,basemap_to_tiles,basemaps,Polygon,GeoJSON
from ipywidgets import Button, HBox, VBox,widgets,Layout,GridspecLayout,IntSlider,HTML
from IPython.display import display
import spatialite
import pandas as pd
import geopandas as gpd
import json
import time
db = spatialite.connect('databases/spatialDB.sqlite')

def stateChanged(slid):
    layer_group.clear_layers()
    stateGeomSql = f"SELECT ST_AsBinary(geom) as geom FROM us_states where name='{states.value}';"
    dominosSql = f"""SELECT ST_AsBinary(s.geom) as geom FROM us_states u,dominos s where u.name='{states.value}'
     and st_contains(u.geom,s.geom) and s.rowid in(SELECT ROWID 
    FROM SpatialIndex
    WHERE f_table_name = 'dominos' 
        AND search_frame = u.geom)"""
    gdf = gpd.GeoDataFrame.from_postgis(stateGeomSql, db,crs = 'EPSG:4269').to_crs('EPSG:4326')
    dominosgdf = gpd.GeoDataFrame.from_postgis(dominosSql, db,crs = 'EPSG:4326')
    center = [gdf.centroid.y.values[0],gdf.centroid.x.values[0]]
    sMap.center = center
    sMap.zoom = 6
    geo_data = GeoJSON(data = json.loads(gdf.to_json()),style={'opacity': 1, 'dashArray': '9', 'fillOpacity': 0, 'weight': 1})
    layer_group.add_layer(geo_data)
    geo_data_dominos = GeoJSON(data = json.loads(dominosgdf.to_json()))
    layer_group.add_layer(geo_data_dominos)
    counts.value = str(len(dominosgdf))
    
sql = "SELECT name FROM us_states order by name;"
statedf = pd.read_sql_query(sql,db)
sMap= Map(center=(41.482222, -81.669722), zoom=15,prefer_canvas =True)
layer_group = LayerGroup()
sMap.add_layer(layer_group)
states = widgets.Dropdown(
    options=statedf.name.values,
    value=statedf.name.values[0],
    description='State:',
    disabled=False,
)
counts=widgets.Text(
    value='',
    placeholder='',
    description='Total:',
    disabled=True,
)
states.observe(stateChanged, 'value')
filterParams=HBox([sMap,VBox([states,counts])])
stateChanged(None)
filterParams

Another example 

**Total dominos in each state**

And this is the query

```sql
   SELECT u.name,count(*) as total_dominos from us_states u,dominos s
 where st_contains(u.geom,s.geom) group by u.name
```

In [ ]:
disp = widgets.Output()
db = spatialite.connect('databases/spatialDB.sqlite')
stateGeomSql = f"""SELECT u.name,count(*) as total_dominos from us_states u,dominos s
 where st_contains(u.geom,s.geom) and s.rowid in(SELECT ROWID 
    FROM SpatialIndex
    WHERE f_table_name = 'dominos' 
        AND search_frame = u.geom) group by u.name"""
data = pd.read_sql_query(stateGeomSql,con=db)
with disp:
    display(data)
disp

One more example

**Total homicides in each neighborhood**

Here are the tables

In [5]:
from ipywidgets import HBox, VBox,widgets,Layout,HTML
from IPython.display import display
db = spatialite.connect('databases/spatialDB.sqlite')
table1 = pd.read_sql_query('select boroname,name,geom as geometry from nyc_neighborhoods limit 5',db)
table2 = pd.read_sql_query('select id,weapon, light_dark, year, geom as geometry from nyc_homicides limit 5',db)
table1_disp = widgets.Output()
table2_disp = widgets.Output()
table1_header = widgets.HTML(value = f"<b><font color='red'><center>NYC_NEIGHBORHOODS</center></b>")
table2_header = widgets.HTML(value = f"<b><font color='red'><center>NYC_HOMICIDES</center></b>")
with table1_disp:
    display(table1)
with table2_disp:
    display(table2)
out=HBox([VBox([table1_header,table1_disp],layout = Layout(margin='0 100px 0 0')),VBox([table2_header,table2_disp])])
out

NameError: name 'spatialite' is not defined

And here is the query

```sql
   SELECT u.boroname,count(*) as total_homicides from nyc_neighborhoods u,nyc_homicides s
 where st_contains(u.geom,s.geom) group by u.boroname
```

In [ ]:
disp = widgets.Output()
db = spatialite.connect('databases/spatialDB.sqlite')
stateGeomSql = f"""SELECT u.boroname,count(*) as total_homicides from nyc_neighborhoods u,nyc_homicides s
 where st_contains(u.geom,s.geom) and s.rowid in(SELECT ROWID 
    FROM SpatialIndex
    WHERE f_table_name = 'nyc_homicides' 
        AND search_frame = u.geom) group by u.boroname"""
data = pd.read_sql_query(stateGeomSql,con=db)
with disp:
    display(data)
disp

One last example

**Total earthquakes in each state**

Let's look at the tables

In [ ]:
from ipywidgets import HBox, VBox,widgets,Layout,HTML
from IPython.display import display
db = spatialite.connect('databases/spatialDB.sqlite')
table1 = pd.read_sql_query('select statefp,name,geom as geometry from us_states limit 5',db)
table2 = pd.read_sql_query('select * from earthquakes limit 5',db)
table1_disp = widgets.Output()
table2_disp = widgets.Output()
table1_header = widgets.HTML(value = f"<b><font color='red'><center>US_STATES</center></b>")
table2_header = widgets.HTML(value = f"<b><font color='red'><center>EARTHQUAKES</center></b>")
with table1_disp:
    display(table1)
with table2_disp:
    display(table2)
out=HBox([VBox([table1_header,table1_disp],layout = Layout(margin='0 100px 0 0')),VBox([table2_header,table2_disp])])
out

And here is the query

```sql
SELECT u.name,count(*) as total_earthquakes from us_states u,earthquakes s
 where st_contains(u.geom,s.geometry) group by u.name
```

In [ ]:
disp = widgets.Output()
db = spatialite.connect('databases/spatialDB.sqlite')
stateGeomSql = f"""SELECT u.name,count(*) as total_earthquakes from us_states u,earthquakes s
 where st_contains(u.geom,s.geometry) and s.rowid in(SELECT ROWID 
    FROM SpatialIndex
    WHERE f_table_name = 'earthquakes' 
        AND search_frame = u.geom) group by u.name"""
data = pd.read_sql_query(stateGeomSql,con=db)
with disp:
    display(data)
disp

Now when we look at the results we might not see any spatial patterns as such. But this is where spatial data and spatial database shines.

To show the spatial distribution we use a particular type of map called **Choropleth map**. 

**A choropleth map showing spatial distribution of earthquakes**

In [ ]:
from branca.colormap import linear
import matplotlib.pyplot as plt
from ipyleaflet import Choropleth
disp = widgets.Output()
db = spatialite.connect('databases/spatialDB.sqlite')
stateDataSql = f"""SELECT u.stusps,count(*) as total_earthquakes from us_states u,earthquakes s
 where st_contains(u.geom,s.geometry) and s.rowid in(SELECT ROWID 
    FROM SpatialIndex
    WHERE f_table_name = 'earthquakes' 
        AND search_frame = u.geom) group by u.stusps"""
dat = pd.read_sql_query(stateDataSql,db)
stateSql = """SELECT stusps,st_asbinary(geom) as geom FROM us_states"""
#df=pd.read_sql_query(sql,db)
df = gpd.read_postgis(stateSql,db)
dat = df[['stusps']].merge(dat,on='stusps',how='left').fillna(0)
dat =  dict(zip(dat['stusps'].tolist(), dat['total_earthquakes'].tolist()))
jsondata = json.loads(df.to_json())
for feature in jsondata['features']:
    feature['id'] = feature['properties']['stusps']
layer = Choropleth(
    geo_data=jsondata,
    choro_data=dat,
    colormap=linear.OrRd_03,
    border_color='black',
    style={'fillOpacity': 0.8, 'dashArray': '5, 5'})
sMap= Map(center=(41.482222, -81.669722), zoom=3,prefer_canvas =True)
sMap.add_layer(layer)
sMap

### Intersection Query

#### Intersects

First we look at how to use intersect function to check whether two geometries intersect.

The function **st_intersects(geometry A,geometry B)** returns true if geometry A and geometry B intersect or touch at atleast a single point

![LineIntersects](supplementary/images/intersect.png) 

A real world example would be to identify the houses that fall with in a hazard zone. We would not only want the houses that are with in the hazard zone but also the houses that has some portion of it inside the hazard zone

![Hazard Zones](supplementary/images/hazard_zones.png) 

Let's look at an interactive example. You can use either the line or the rectangle tool to draw your own geometry. The geometries (in this cases us_states) that intersects with the geometry you have drawn will be highlighted in green color. 

In [ ]:
from ipyleaflet import Map,DrawControl,GeoJSON,LayerGroup
import geojson
import json
from shapely.geometry import shape
import geopandas as gpd
import numpy as np
import time
matching=[]
def handle_draw(target, action, geo_json):
    global matching
    draw_group.clear_layers()
    geo_dat = GeoJSON(data = geo_json)
    draw_group.add_layer(geo_dat)
    dc.clear()
    g1 = geojson.loads(json.dumps(geo_json['geometry']))
    g2 = shape(g1)
    sql = f"SELECT stusps from us_states u where st_intersects(u.geom,ST_TRANSFORM(ST_GeomFromText('{g2.wkt}',4326),4269))"
    pdf = pd.read_sql_query(sql, db)
    matching = pdf.values
    for layer in stateGroup.layers:
        layer.style={'weight':2+np.random.random_sample()}
    
def styleChange(feature):
    props=feature['properties']
    if props['stusps'] in matching:
        return {'opacity': 1, 'fillOpacity': 0, 'weight': 1,'color':'green'}
    else:
        return {'opacity': 0.3, 'fillOpacity': 0, 'weight': 1,'color':'red'}

sMap= Map(center=(44.967243, -103.771556), zoom=8,prefer_canvas =True)
sMap.fit_bounds(((24.1, -126.1), (49.9, -64.4)))
dc = DrawControl(
    marker={},
    rectangle={"shapeOptions": {"color": "#0000FF",'fillOpacity':0}},
    circle={},
    circlemarker={},
    polygon={}
)
dc.on_draw(handle_draw)
sMap.add_control(dc)
draw_group = LayerGroup()
sMap.add_layer(draw_group)
stateGroup = LayerGroup()
sMap.add_layer(stateGroup)
stateGeomSql = f"SELECT stusps,ST_AsBinary(geom) as geom FROM us_states;"
gdf = gpd.GeoDataFrame.from_postgis(stateGeomSql, db,crs = 'EPSG:4269').to_crs('EPSG:4326')
sMap.zoom = 6
geo_data = GeoJSON(data = json.loads(gdf.to_json()),style={'opacity': 0.3, 'fillOpacity': 0, 'weight': 1,'color':'red'},style_callback=styleChange)
stateGroup.add_layer(geo_data)
sMap

#### Intersection

While intersects check whether two geometries intersect, intersection returns the geometry shared by the two geometries.

The function **st_intersection(geometry A,geometry B)** returns the geometry shared by geometry A and geometry B

![LineIntersection](supplementary/images/intersection.png) 

Now we will look at a concrete example of using st_intersection

**Find total length of subway line in each neighborhood**

![RoadIntersection](supplementary/images/intersection_example.png) 

Here we will use one more function **st_length(geometry)** for calculating the length of a geometry

The function **st_length(geometry A)** returns the length of geometry A

The tables involved in this query

In [ ]:
from ipywidgets import HBox, VBox,widgets,Layout,HTML
from IPython.display import display
db = spatialite.connect('databases/spatialDB.sqlite')
table1 = pd.read_sql_query('select boroname,name,geom as geometry from nyc_neighborhoods limit 5',db)
table2 = pd.read_sql_query('select pk_uid,geometry from nyc_subway_lines limit 5',db)
table1_disp = widgets.Output()
table2_disp = widgets.Output()
table1_header = widgets.HTML(value = f"<b><font color='red'><center>NYC_NEIGHBORHOODS</center></b>")
table2_header = widgets.HTML(value = f"<b><font color='red'><center>NYC_SUBWAY_LINES</center></b>")
with table1_disp:
    display(table1)
with table2_disp:
    display(table2)
out=HBox([VBox([table1_header,table1_disp],layout = Layout(margin='0 100px 0 0')),VBox([table2_header,table2_disp])])
out

The required query

```sql
SELECT u.boroname,sum(ST_Length(st_intersection(u.geom,s.geometry))) as total_length from nyc_neighborhoods u,nyc_subway_lines s
 where st_intersects(u.geom,s.geometry) group by u.boroname
```

In [ ]:
disp = widgets.Output()
db = spatialite.connect('databases/spatialDB.sqlite')
stateGeomSql = f"""SELECT u.boroname,sum(ST_Length(st_intersection(u.geom,s.geometry))) as total_length from nyc_neighborhoods u,nyc_subway_lines s
 where st_intersects(u.geom,s.geometry) and s.rowid in(SELECT ROWID 
    FROM SpatialIndex
    WHERE f_table_name = 'nyc_subway_lines' 
        AND search_frame = u.geom) group by u.boroname"""
data = pd.read_sql_query(stateGeomSql,con=db)
with disp:
    display(data)
disp

### With in a Distance Queries

With in distance queries are used to find out geometrical objects that are with in a specific distance of a particular geometrical object.

![Withindistance_example](supplementary/images/withindistance.png)

![Withindistance](supplementary/images/distance_within_example.png) 

#### Buffer

![Buffer](supplementary/images/buffer.png) 

The function **st_buffer(geometry A,distance)**  encircles geometry A at a specified **distance** and returns a geometry object that is the buffer that surrounds the source object (A).

Lets look at a concrete example 

**Number of homicides with in 100 meter radius of NYC substations**

The required tables are

In [ ]:
from ipywidgets import HBox, VBox,widgets,Layout,HTML
from IPython.display import display
db = spatialite.connect('databases/spatialDB.sqlite')
table1 = pd.read_sql_query('select pk_uid,name,geom as geometry from nyc_substations limit 5',db)
table2 = pd.read_sql_query('select pk_uid,weapon,year,geom as geometry from nyc_homicides limit 5',db)
table1_disp = widgets.Output()
table2_disp = widgets.Output()
table1_header = widgets.HTML(value = f"<b><font color='red'><center>NYC_SUBSTATIONS</center></b>")
table2_header = widgets.HTML(value = f"<b><font color='red'><center>NYC_HOMICIDES</center></b>")
with table1_disp:
    display(table1)
with table2_disp:
    display(table2)
out=HBox([VBox([table1_header,table1_disp],layout = Layout(margin='0 100px 0 0')),VBox([table2_header,table2_disp])])
out

And the query is 

```sql
SELECT u.name,count(*) as total_homicides from nyc_substations u,
nyc_homicides s where st_contains(st_buffer(u.geom,100),s.geom) group by u.name
```

In [ ]:
disp = widgets.Output()
db = spatialite.connect('databases/spatialDB.sqlite')
stateGeomSql = f"""SELECT u.name,count(*) as total_homicides from (select name,st_buffer(geom,100) as geom from nyc_substations) u,
nyc_homicides s where st_contains(u.geom,s.geom) and s.rowid in(SELECT ROWID 
    FROM SpatialIndex
    WHERE f_table_name = 'nyc_homicides' 
        and f_geometry_column = 'geom'
        AND search_frame = u.geom)  group by u.name order by count(*) desc"""
data = pd.read_sql_query(stateGeomSql,con=db)
with disp:
    display(data)
disp

Let's look at another example 

**Number of shooting incidents with in 50 meter of schools**

and the tables are 

In [ ]:
from ipywidgets import HBox, VBox,widgets,Layout,HTML
from IPython.display import display
import spatialite
import pandas as pd
db = spatialite.connect('databases/spatialDB.sqlite')
table1 = pd.read_sql_query('select pk_uid,schoolname,sch_type,geometry from nyc_schools limit 5',db)
table2 = pd.read_sql_query('select pk_uid,geometry from nyc_shooting limit 5',db)
table1_disp = widgets.Output()
table2_disp = widgets.Output()
table1_header = widgets.HTML(value = f"<b><font color='red'><center>NYC_SCHOOLS</center></b>")
table2_header = widgets.HTML(value = f"<b><font color='red'><center>NYC_SHOOTING</center></b>")
with table1_disp:
    display(table1)
with table2_disp:
    display(table2)
out=HBox([VBox([table1_header,table1_disp],layout = Layout(margin='0 100px 0 0')),VBox([table2_header,table2_disp])])
out

And the query

```sql
SELECT a.schoolname,count(*) as total_shooting_incidents from nyc_shooting u,nyc_schools a
 where st_contains(st_buffer(a.geom,50),u.geometry) group by a.schoolname
```

In [ ]:
disp = widgets.Output()
db = spatialite.connect('databases/spatialDB.sqlite')
stateGeomSql = f"""WITH a AS (select schoolname,st_buffer(geometry,50) as geom from nyc_schools)
SELECT a.schoolname,count(*) as total_shooting_incidents from nyc_shooting u,a
 where st_contains(a.geom,u.geometry) and u.rowid in(SELECT ROWID 
    FROM SpatialIndex
    WHERE f_table_name = 'nyc_shooting' 
        and f_geometry_column = 'geometry'
        AND search_frame = a.geom)   group by a.schoolname order by count(*) desc"""
data = pd.read_sql_query(stateGeomSql,con=db)
with disp:
    display(data)
disp

Let's look at another example

**How many hospitals are there with in a specific distance of where you are**

The required tables

In [ ]:
from ipywidgets import HBox, VBox,widgets,Layout,HTML
from IPython.display import display
import spatialite
import pandas as pd
db = spatialite.connect('databases/spatialDB.sqlite')
table1 = pd.read_sql_query('select pk_uid,geom as geometry from hospitals limit 5',db)
table1_disp = widgets.Output()
table1_header = widgets.HTML(value = f"<b><font color='red'><center>HOSPITALS</center></b>")
with table1_disp:
    display(table1)
out=HBox([VBox([table1_header,table1_disp],layout = Layout(margin='0 100px 0 0'))])
out

Let's look at an interactive example. Double click anywhere on the map and it will be selected as the current location. Based on the slider value selected (default is 3000 meter). You can change the slider to change the buffer value

In [ ]:
from ipyleaflet import Map, DrawControl,GeoData,LayerGroup,Polygon,GeoJSON,Marker
from ipywidgets import Button, HBox, VBox,widgets,Layout,GridspecLayout,IntSlider,HTML
from IPython.display import display
import spatialite
import pandas as pd
import geopandas as gpd
import json
import time
db = spatialite.connect('databases/spatialDB.sqlite')
coords=None

def handle_click(**kwargs):
    global coords
    if kwargs.get('type') == 'dblclick':
        layer_group.clear_layers()
        coords = kwargs.get('coordinates')
        layer_group.add_layer(Marker(location=coords))
        findHospitals()
        
def findHospitals():
    global coords
    if coords is not None:
        stateGeomSql = f"""SELECT st_asbinary(u.geom) as geom from hospitals u
 where st_contains(st_buffer(st_transform(MakePoint({coords[1]},{coords[0]},4326),3857),{radiusSlider.value}),
 st_transform(u.geom,3857))"""
        gdf = gpd.GeoDataFrame.from_postgis(stateGeomSql, db,crs = 'EPSG:4326')
        if len(gdf)!=0:
            
            geo_data = GeoData(geo_dataframe = gdf,
                style={'color': 'black', 'radius':8, 'fillColor': '#3366cc', 'opacity':0.5, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                point_style={'radius': 5, 'color': 'red', 'fillOpacity': 0.8, 'fillColor': 'blue', 'weight': 3},
                name = 'Release')
            layer_group.add_layer(geo_data)
            center = [gdf.centroid.y.values[0],gdf.centroid.x.values[0]]
            sMap.center = center
            sMap.zoom = 12

def radiusChanged(slider):
    findHospitals()
        
sMap= Map(center=(41.482222, -81.669722), zoom=15,prefer_canvas =True)

radiusSlider = widgets.IntSlider(
    value=3000,
    min=0,
    max=100000,
    step=500,
    description='Radius:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

radiusSlider.observe(radiusChanged, 'value')
layer_group = LayerGroup()
sMap.add_layer(layer_group)
sMap.on_interaction(handle_click)
filterParams=HBox([sMap,VBox([radiusSlider])])
filterParams

### Distance Function

The distance function is used to find the distance between two geometries

The function **st_distance(geometry A,geometry B)** returns the distance between geometry A and geometry B

![DistanceRestaurant](supplementary/images/distance_restaurant_example.png) 

Let's look at an example 

**Show five nearest tourist place from your current location**

The required tables

In [ ]:
from ipywidgets import HBox, VBox,widgets,Layout,HTML
from IPython.display import display
import spatialite
import pandas as pd
db = spatialite.connect('databases/spatialDB.sqlite')
table1 = pd.read_sql_query('select pk_uid,geom as geometry from toUrism limit 5',db)
table1_disp = widgets.Output()
table1_header = widgets.HTML(value = f"<b><font color='red'><center>TOURISM</center></b>")
with table1_disp:
    display(table1)
out=HBox([VBox([table1_header,table1_disp],layout = Layout(margin='0 100px 0 0'))])
out

In [ ]:
from ipyleaflet import Map, DrawControl,GeoData,LayerGroup,Polygon,GeoJSON,Marker
from ipywidgets import Button, HBox, VBox,widgets,Layout,GridspecLayout,IntSlider,HTML
from IPython.display import display
import spatialite
import pandas as pd
import geopandas as gpd
import json
import time
db = spatialite.connect('databases/spatialDB.sqlite')
coords=None

def handle_click(**kwargs):
    global coords
    if kwargs.get('type') == 'dblclick':
        layer_group.clear_layers()
        coords = kwargs.get('coordinates')
        layer_group.add_layer(Marker(location=coords))
        findNearestTouristPlaces()
        
def findNearestTouristPlaces():
    global coords
    if coords is not None:
        stateGeomSql = f"""SELECT u.pk_uid,st_asbinary(u.geom) as geom,st_distance(st_transform(u.geom,3857),
        st_transform(MakePoint({coords[1]},{coords[0]},4326),3857)) as 
        dist_to_tourist from tourism u order by dist_to_tourist asc limit 5"""
        gdf = gpd.GeoDataFrame.from_postgis(stateGeomSql, db,crs = 'EPSG:4326')
        geo_data = GeoData(geo_dataframe = gdf,
            style={'color': 'black', 'radius':8, 'fillColor': '#3366cc', 'opacity':0.5, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
            hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
            point_style={'radius': 5, 'color': 'red', 'fillOpacity': 0.8, 'fillColor': 'blue', 'weight': 3},
            name = 'Release')
        layer_group.add_layer(geo_data)
        center = [gdf.centroid.y.values[0],gdf.centroid.x.values[0]]
        sMap.center = center
        sMap.zoom = 12

sMap= Map(center=(41.482222, -81.669722), zoom=15,prefer_canvas =True)
layer_group = LayerGroup()
sMap.add_layer(layer_group)
sMap.on_interaction(handle_click)
sMap

And one final example

**Show five nearest gas stations from your current location**

The required table

In [ ]:
from ipywidgets import HBox, VBox,widgets,Layout,HTML
from IPython.display import display
import spatialite
import pandas as pd
db = spatialite.connect('databases/spatialDB.sqlite')
table1 = pd.read_sql_query('select pk_uid,geom from gas_stationS limit 5',db)
table1_disp = widgets.Output()
table1_header = widgets.HTML(value = f"<b><font color='red'><center>GAS_STATIONS</center></b>")
with table1_disp:
    display(table1)
out=HBox([VBox([table1_header,table1_disp],layout = Layout(margin='0 100px 0 0'))])
out

In [ ]:
from ipyleaflet import Map, DrawControl,GeoData,LayerGroup,Polygon,GeoJSON,Marker
from ipywidgets import Button, HBox, VBox,widgets,Layout,GridspecLayout,IntSlider,HTML
from IPython.display import display
import spatialite
import pandas as pd
import geopandas as gpd
import json
import time
db = spatialite.connect('databases/spatialDB.sqlite')
coords=None

def handle_click(**kwargs):
    global coords
    if kwargs.get('type') == 'dblclick':
        layer_group.clear_layers()
        coords = kwargs.get('coordinates')
        layer_group.add_layer(Marker(location=coords))
        findNearestGasStation()
        
def findNearestGasStation():
    global coords
    if coords is not None:
        stateGeomSql = f"""SELECT u.pk_uid,st_asbinary(u.geom) as geom,st_distance(u.geom,
        MakePoint({coords[1]},{coords[0]},4326)) as 
        dist_to_gas_station from gas_stations u order by dist_to_gas_station asc limit 5"""
        gdf = gpd.GeoDataFrame.from_postgis(stateGeomSql, db,crs = 'EPSG:4326')
        geo_data = GeoData(geo_dataframe = gdf,
            style={'color': 'black', 'radius':8, 'fillColor': '#3366cc', 'opacity':0.5, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
            hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
            point_style={'radius': 5, 'color': 'red', 'fillOpacity': 0.8, 'fillColor': 'blue', 'weight': 3},
            name = 'Release')
        layer_group.add_layer(geo_data)
        center = [gdf.centroid.y.values[0],gdf.centroid.x.values[0]]
        sMap.center = center
        sMap.zoom = 12
        
sMap= Map(center=(41.482222, -81.669722), zoom=15,prefer_canvas =True)
layer_group = LayerGroup()
sMap.add_layer(layer_group)
sMap.on_interaction(handle_click)
sMap